In [1]:
import sys
sys.path.append('../src')
from order_book import Book
from order_book import Format
from event import Event
from datetime import datetime
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio


# import lobster message file
cols = ['time', 'type', 'id', 'shares', 'price', 'direction']
data = pd.read_csv("../data/lobster/AAPL_2012-06-21_34200000_37800000_message_50.csv", names=cols)
# re-scale the price col
data.price = data.price/10000
# make sure data is during market hours
data = data[data['time']>= 9.5*60*60]
data = data[data['time']<= 16*60*60]
print(len(data))
data.head()

91997


,time,type,id,shares,price,direction
0,34200.004241,1,16113575,18,585.33,1
1,34200.004261,1,16113584,18,585.32,1
2,34200.004447,1,16113594,18,585.31,1
3,34200.025552,1,16120456,18,585.91,-1
4,34200.025580,1,16120480,18,585.92,-1


In [2]:
# create order book object
book = Book()
events = []

# iterate through event messages and read into book - test using first 20k events
for i in range(150):
    event = Event(data.loc[i])
    book.handleEvent(event, i)
    events.append(event)

In [3]:
# pull all the visible executions into a trades data frame
# split them into buys and sells based on direction
trades = pd.DataFrame(book.trades, columns=['Time', 'Price', 'Shares', 'Direction'])
trades.Time = [datetime.fromtimestamp(trades.iloc[x].Time).time() for x in range(len(trades))]
sells = trades[trades['Direction']==-1]
buys = trades[trades['Direction']==1]

In [4]:
# pull all the hidden executions into a trades data frame
# split them into buys and sells based on direction
hidden_trades = pd.DataFrame(book.hidden_trades, columns=['Time', 'Price', 'Shares', 'Direction'])
hidden_trades.Time = [datetime.fromtimestamp(hidden_trades.iloc[x].Time).time() for x in range(len(hidden_trades))]
hidden_sells = hidden_trades[hidden_trades['Direction']==-1]
hidden_buys = hidden_trades[hidden_trades['Direction']==1]

In [5]:
# get the book snapshots list and format it to match the LOBSTER output
# We split the bid and ask side of the book to make it easier to plot below
# remove the first few rows as they are 0
formatted_book = Format(book.book_snapshot, 100).formatBook()
# split the bids and asks starting from a defined index to remove the zero values at the begining 
bids, asks = formatted_book.splitBidsAsks()


In [6]:
# take the bid and ask data
bid_ask_data = formatted_book.data
bid_ask_data = bid_ask_data.reset_index().rename(columns={'index':'Time'})
bid_ask_data = bid_ask_data.drop_duplicates()
# sync the starting time stamp with the trades data
trades = trades[trades['Time']>=bid_ask_data.iloc[0]['Time']]

# now lets trim the time stamps to show only seconds and smaller
bid_ask_data.Time = [datetime.strptime(str(i), "%H:%M:%S.%f").strftime("%S.%f") for i in bid_ask_data.Time]
trades.Time = [datetime.strptime(str(i), "%H:%M:%S.%f").strftime("%S.%f") for i in trades.Time]

In [7]:
# set up fig dict
my_fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

In [8]:
# fill in most of layout
my_fig_dict["layout"]["xaxis"] = {"autorange": True, 'tickmode':'linear', 'dtick':'250ms', 'tickformat':'%S.%.3f'}
my_fig_dict["layout"]["yaxis"] = {"title": "AAPL Price"}
my_fig_dict["layout"]["hovermode"] = "closest"
my_fig_dict["layout"]["barmode"] = "stack"
my_fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {"args": [None, {"frame": {"duration": 0, "redraw": False},"fromcurrent": True, "transition": {"duration": 0,"easing": "quadratic-in-out"}}],"label": "Play","method": "animate"},
            {"args": [[None], {"frame": {"duration": 0, "redraw": False},"mode": "immediate","transition": {"duration": 0}}],"label": "Pause","method": "animate"}
        ]
    }
]

my_sliders_dict = {"active": 0,"xanchor": "left","currentvalue": {"prefix": "Time:",},"pad": {"b": 0, "t": 120},"len": 0.9,"x": 0.05,"y": 0,"steps": []}

In [9]:
# create all the traces then add to the dictionary
best_bid_price_trace = dict(x=bid_ask_data.Time, y=bid_ask_data.Bid_1, mode='lines', line=dict(color='green'), name='best_bid')
my_fig_dict["data"].append(best_bid_price_trace)

best_ask_price_trace = dict(x=bid_ask_data.Time, y=bid_ask_data.Ask_1, mode='lines', line=dict(color='red'), name='best_ask')
my_fig_dict["data"].append(best_ask_price_trace)

best_bid_vol_trace = go.Bar(x=bid_ask_data.Time, y=bid_ask_data.Bid_1_Vol, marker=dict(color='green'), name='best_bid_vol', text=bid_ask_data.Bid_1.astype(str))
my_fig_dict["data"].append(best_bid_vol_trace)

best_ask_vol_trace = go.Bar(x=bid_ask_data.Time, y=bid_ask_data.Ask_1_Vol, marker=dict(color='red'), name='best_ask_vol', text=bid_ask_data.Ask_1.astype(str))
my_fig_dict["data"].append(best_ask_vol_trace)

trades_trace = dict(x=trades.Time, y=trades.Price, mode='markers', marker=dict(color='blue', size=np.log(trades.Shares)*5), name='Executions', text='Shares: ' + trades.Shares.astype(str))
my_fig_dict["data"].append(trades_trace)

In [10]:
# create frames, adding each to frame dict

for time_step in bid_ask_data.Time:
    my_frame ={'data': [], 'name': str(time_step)}

    time_step_data = bid_ask_data[bid_ask_data['Time'] <= time_step]
    time_step_trades_data = trades[trades['Time'] <= time_step]
    
    best_bid_price_time_trace = dict(x=time_step_data.Time, y=time_step_data.Bid_1, mode='lines', line=dict(color='green'), name='best_bid')
    my_frame["data"].append(best_bid_price_time_trace)

    best_ask_price_time_trace = dict(x=time_step_data.Time, y=time_step_data.Ask_1, mode='lines', line=dict(color='red'), name='best_ask')
    my_frame["data"].append(best_ask_price_time_trace)

    best_bid_vol_time_trace = go.Bar(x=time_step_data.Time, y=time_step_data.Bid_1_Vol, marker=dict(color='green'), name='best_bid_vol', text=time_step_data.Bid_1.astype(str))
    my_frame["data"].append(best_bid_vol_time_trace)

    best_ask_vol_time_trace = go.Bar(x=time_step_data.Time, y=time_step_data.Ask_1_Vol, marker=dict(color='red'), name='best_ask_vol', text=time_step_data.Ask_1.astype(str))
    my_frame["data"].append(best_ask_vol_time_trace)

    trades_time_trace = dict(x=time_step_trades_data.Time, y=time_step_trades_data.Price, mode='markers', marker=dict(color='blue', size=np.log(trades.Shares)*5), name='Executions', text='Shares: ' + trades.Shares.astype(str))
    my_frame["data"].append(trades_time_trace)

    my_fig_dict["frames"].append(my_frame)
    my_slider_step = {"args": [[time_step],{"frame": {"duration": 0, "redraw": False},"mode": "immediate","transition": {"duration": 0}}],"label": str(time_step),"method": "animate"}
    my_sliders_dict["steps"].append(my_slider_step)

In [11]:
# present data
my_fig_dict["layout"]["sliders"] = [my_sliders_dict]

# feed all data into plotly
fig = go.Figure(my_fig_dict).set_subplots(2, 1, subplot_titles=('Top of Book Price', 'Top of Book Volume'), vertical_spacing=0.05, row_heights=[0.6, 0.4], shared_xaxes=True)
# set x and y for volume bar chart
fig.data[2].xaxis = "x2"
fig.data[2].yaxis = "y2"
fig.data[3].xaxis = "x2"
fig.data[3].yaxis = "y2"
fig.update_layout(yaxis2_title='Shares', xaxis2_title='Time')
fig.update_layout(width=1440, height=722, margin=dict(l=50, r=50, b=75, t=50), yaxis2_range=[0,250])
fig.show()
#fig.write_html("../data/html/AAPL_L1_Data.html")


In [12]:
# write to gif 
import io
import PIL
frames = []
for s, fr in enumerate(fig.frames):
    # set main traces to appropriate traces within plotly frame
    fig.update(data=fr.data)
    # move slider to correct place
    fig.layout.sliders[0].update(active=s)
    # generate image of current state
    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))

frames[0].save(
        "../data/gif/bid_ask_vol.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=500,
        loop=0,
    )

!['bid_ask_vol.gif'](../data/gif/bid_ask_vol.gif)